In [1]:
import torch
from tsai.models.utils import count_parameters
from prettytable import PrettyTable
import tsai.all

In [2]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

input_features_length = 15
seq_len = 5

def count_model_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params += params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

%run ../shared_functions.py
%run ../my_shared_functions.py

Selected device is cuda


<Figure size 640x480 with 0 Axes>

### FCN

In [30]:
fcn_model = tsai.all.FCN(
    c_in=input_features_length, 
    c_out=1,
    layers=[64, 128, 64],
    kss=[7, 5 ,3]
    )
_ = count_model_parameters(fcn_model)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
| convblock1.0.weight |    6720    |
| convblock1.1.weight |     64     |
|  convblock1.1.bias  |     64     |
| convblock2.0.weight |   40960    |
| convblock2.1.weight |    128     |
|  convblock2.1.bias  |    128     |
| convblock3.0.weight |   24576    |
| convblock3.1.weight |     64     |
|  convblock3.1.bias  |     64     |
|      fc.weight      |     64     |
|       fc.bias       |     1      |
+---------------------+------------+
Total Trainable Params: 72833


In [15]:
count_parameters(fcn_model)

72833

### gMLP

In [32]:
gmlp_model = tsai.all.gMLP(
    input_features_length,
    1,
    seq_len,
    patch_size=1,
    d_model=128,
    d_ffn=256,
    depth=7
    )
_ = count_model_parameters(gmlp_model)

+---------------------------------+------------+
|             Modules             | Parameters |
+---------------------------------+------------+
|       model.0.norm.weight       |    128     |
|        model.0.norm.bias        |    128     |
|   model.0.channel_proj1.weight  |   65536    |
|    model.0.channel_proj1.bias   |    512     |
|   model.0.channel_proj2.weight  |   32768    |
|    model.0.channel_proj2.bias   |    128     |
|     model.0.sgu.norm.weight     |    256     |
|      model.0.sgu.norm.bias      |    256     |
| model.0.sgu.spatial_proj.weight |     25     |
|  model.0.sgu.spatial_proj.bias  |     5      |
|       model.1.norm.weight       |    128     |
|        model.1.norm.bias        |    128     |
|   model.1.channel_proj1.weight  |   65536    |
|    model.1.channel_proj1.bias   |    512     |
|   model.1.channel_proj2.weight  |   32768    |
|    model.1.channel_proj2.bias   |    128     |
|     model.1.sgu.norm.weight     |    256     |
|      model.1.sgu.n

### GRU-FCN

In [33]:
gru_fcn_model = tsai.all.GRU_FCN(
    c_in=input_features_length,
    c_out=1,
    seq_len=seq_len,
    hidden_size=100,
    rnn_layers=2,
    bias=True,
    cell_dropout=0.2,
    rnn_dropout=0.4,
    bidirectional=False,
    fc_dropout=0.2,
    conv_layers=[64, 128, 64],
    kss=[5, 5, 3],
    se=0
    )
_ = count_model_parameters(gru_fcn_model)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|   rnn.weight_ih_l0  |    1500    |
|   rnn.weight_hh_l0  |   30000    |
|    rnn.bias_ih_l0   |    300     |
|    rnn.bias_hh_l0   |    300     |
|   rnn.weight_ih_l1  |   30000    |
|   rnn.weight_hh_l1  |   30000    |
|    rnn.bias_ih_l1   |    300     |
|    rnn.bias_hh_l1   |    300     |
| convblock1.0.weight |    4800    |
| convblock1.1.weight |     64     |
|  convblock1.1.bias  |     64     |
| convblock2.0.weight |   40960    |
| convblock2.1.weight |    128     |
|  convblock2.1.bias  |    128     |
| convblock3.0.weight |   24576    |
| convblock3.1.weight |     64     |
|  convblock3.1.bias  |     64     |
|      fc.weight      |    164     |
|       fc.bias       |     1      |
+---------------------+------------+
Total Trainable Params: 163713


### GRU

In [34]:
gru_model = tsai.all.GRU(
                c_in=input_features_length,
                c_out=1,
                hidden_size=100,
                n_layers=2,
                rnn_dropout=0.2,
                fc_dropout=0.4)
_ = count_model_parameters(gru_model)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| rnn.weight_ih_l0 |    4500    |
| rnn.weight_hh_l0 |   30000    |
|  rnn.bias_ih_l0  |    300     |
|  rnn.bias_hh_l0  |    300     |
| rnn.weight_ih_l1 |   30000    |
| rnn.weight_hh_l1 |   30000    |
|  rnn.bias_ih_l1  |    300     |
|  rnn.bias_hh_l1  |    300     |
|    fc.weight     |    100     |
|     fc.bias      |     1      |
+------------------+------------+
Total Trainable Params: 95801


### InceptionTime

In [35]:
inceptiontime_model = tsai.all.InceptionTime(
    c_in=input_features_length,
    c_out=1,
    nf=64,
    nb_filters=None)
_ = count_model_parameters(inceptiontime_model)

+-------------------------------------------------+------------+
|                     Modules                     | Parameters |
+-------------------------------------------------+------------+
|   inceptionblock.inception.0.bottleneck.weight  |    960     |
|    inceptionblock.inception.0.convs.0.weight    |   159744   |
|    inceptionblock.inception.0.convs.1.weight    |   77824    |
|    inceptionblock.inception.0.convs.2.weight    |   36864    |
| inceptionblock.inception.0.maxconvpool.1.weight |    960     |
|       inceptionblock.inception.0.bn.weight      |    256     |
|        inceptionblock.inception.0.bn.bias       |    256     |
|   inceptionblock.inception.1.bottleneck.weight  |   16384    |
|    inceptionblock.inception.1.convs.0.weight    |   159744   |
|    inceptionblock.inception.1.convs.1.weight    |   77824    |
|    inceptionblock.inception.1.convs.2.weight    |   36864    |
| inceptionblock.inception.1.maxconvpool.1.weight |   16384    |
|       inceptionblock.in

In [6]:
# number of parameters for default HP
inceptiontime_model = tsai.all.InceptionTime(
    c_in=input_features_length,
    c_out=1)
_ = count_model_parameters(inceptiontime_model)

+-------------------------------------------------+------------+
|                     Modules                     | Parameters |
+-------------------------------------------------+------------+
|   inceptionblock.inception.0.bottleneck.weight  |    480     |
|    inceptionblock.inception.0.convs.0.weight    |   39936    |
|    inceptionblock.inception.0.convs.1.weight    |   19456    |
|    inceptionblock.inception.0.convs.2.weight    |    9216    |
| inceptionblock.inception.0.maxconvpool.1.weight |    480     |
|       inceptionblock.inception.0.bn.weight      |    128     |
|        inceptionblock.inception.0.bn.bias       |    128     |
|   inceptionblock.inception.1.bottleneck.weight  |    4096    |
|    inceptionblock.inception.1.convs.0.weight    |   39936    |
|    inceptionblock.inception.1.convs.1.weight    |   19456    |
|    inceptionblock.inception.1.convs.2.weight    |    9216    |
| inceptionblock.inception.1.maxconvpool.1.weight |    4096    |
|       inceptionblock.in

### LSTM-FCN

In [36]:
lstm_fcn_model = tsai.all.LSTM_FCN(
                c_in=input_features_length, 
                c_out=1,
                seq_len=seq_len,
                hidden_size=100,
                rnn_layers=2,
                bias=True,
                cell_dropout=0.2,
                rnn_dropout=0,
                bidirectional=False,
                fc_dropout=0.2,
                conv_layers=[64, 128, 64],
                kss=[5, 5, 3],
                se=0
                )
_ = count_model_parameters(lstm_fcn_model)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|   rnn.weight_ih_l0  |    2000    |
|   rnn.weight_hh_l0  |   40000    |
|    rnn.bias_ih_l0   |    400     |
|    rnn.bias_hh_l0   |    400     |
|   rnn.weight_ih_l1  |   40000    |
|   rnn.weight_hh_l1  |   40000    |
|    rnn.bias_ih_l1   |    400     |
|    rnn.bias_hh_l1   |    400     |
| convblock1.0.weight |    4800    |
| convblock1.1.weight |     64     |
|  convblock1.1.bias  |     64     |
| convblock2.0.weight |   40960    |
| convblock2.1.weight |    128     |
|  convblock2.1.bias  |    128     |
| convblock3.0.weight |   24576    |
| convblock3.1.weight |     64     |
|  convblock3.1.bias  |     64     |
|      fc.weight      |    164     |
|       fc.bias       |     1      |
+---------------------+------------+
Total Trainable Params: 194613


### LSTM

In [37]:
lstm_model = tsai.all.LSTM(
                c_in=input_features_length,
                c_out=1,
                hidden_size=100,
                n_layers=2,
                rnn_dropout=0,
                fc_dropout=0)
_ = count_model_parameters(lstm_model)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| rnn.weight_ih_l0 |    6000    |
| rnn.weight_hh_l0 |   40000    |
|  rnn.bias_ih_l0  |    400     |
|  rnn.bias_hh_l0  |    400     |
| rnn.weight_ih_l1 |   40000    |
| rnn.weight_hh_l1 |   40000    |
|  rnn.bias_ih_l1  |    400     |
|  rnn.bias_hh_l1  |    400     |
|    fc.weight     |    100     |
|     fc.bias      |     1      |
+------------------+------------+
Total Trainable Params: 127701


### MLSTM-FCN

In [38]:
mlstm_fcn_model = tsai.all.MLSTM_FCN(
    c_in=input_features_length, 
    c_out=1,
    seq_len=seq_len,
    hidden_size=100,
    rnn_layers=1,
    bias=True,
    cell_dropout=0.2,
    rnn_dropout=0.4,
    bidirectional=False,
    fc_dropout=0.2,
    conv_layers=[64, 256, 64],
    kss=[5, 3, 1],
    se=16
    )
_ = count_model_parameters(mlstm_fcn_model)

+---------------------+------------+
|       Modules       | Parameters |
+---------------------+------------+
|   rnn.weight_ih_l0  |    2000    |
|   rnn.weight_hh_l0  |   40000    |
|    rnn.bias_ih_l0   |    400     |
|    rnn.bias_hh_l0   |    400     |
| convblock1.0.weight |    4800    |
| convblock1.1.weight |     64     |
|  convblock1.1.bias  |     64     |
|   se1.fc.0.weight   |    256     |
|   se1.fc.2.weight   |    256     |
| convblock2.0.weight |   49152    |
| convblock2.1.weight |    256     |
|  convblock2.1.bias  |    256     |
|   se2.fc.0.weight   |    4096    |
|   se2.fc.2.weight   |    4096    |
| convblock3.0.weight |   16384    |
| convblock3.1.weight |     64     |
|  convblock3.1.bias  |     64     |
|      fc.weight      |    164     |
|       fc.bias       |     1      |
+---------------------+------------+
Total Trainable Params: 122773


### ResCNN

In [39]:
rescnn_model = tsai.all.ResCNN(
                c_in=input_features_length, 
                c_out=1,
                coord=False,
                separable=False,
                zero_norm=False
            )
_ = count_model_parameters(rescnn_model)

+----------------------------+------------+
|          Modules           | Parameters |
+----------------------------+------------+
| block1.convblock1.0.weight |    6720    |
| block1.convblock1.1.weight |     64     |
|  block1.convblock1.1.bias  |     64     |
| block1.convblock2.0.weight |   20480    |
| block1.convblock2.1.weight |     64     |
|  block1.convblock2.1.bias  |     64     |
| block1.convblock3.0.weight |   12288    |
| block1.convblock3.1.weight |     64     |
|  block1.convblock3.1.bias  |     64     |
|  block1.shortcut.0.weight  |    960     |
|  block1.shortcut.1.weight  |     64     |
|   block1.shortcut.1.bias   |     64     |
|      block2.0.weight       |   24576    |
|      block2.1.weight       |    128     |
|       block2.1.bias        |    128     |
|      block3.0.weight       |   98304    |
|      block3.1.weight       |    256     |
|       block3.1.bias        |    256     |
|      block3.2.weight       |     1      |
|      block4.0.weight       |  

### TSiT

In [41]:
tsit_model = tsai.all.TSiT(
    c_in=input_features_length,
    c_out=1,
    seq_len=seq_len,
    d_model=32,
    depth=5,
    n_heads=16,
    act='gelu',
    lsa=False,
    attn_dropout=0.3,
    dropout=0,
    drop_path_rate=0,
    mlp_ratio=1,
    qkv_bias=True,
    pre_norm=True,
    use_token=False, # use_token set to False as c_out == 1
    use_pe=True,
    cat_pos=None,
    n_cat_embeds=None,
    cat_embed_dims=None,
    cat_padding_idxs=None,
    token_size=None,
    tokenizer=None,
    feature_extractor=None,
    flatten=False,
    concat_pool=True,
    fc_dropout=0.1,
    use_bn=False,
    bias_init=None,
    y_range=None,
    custom_head=None,
)
_ = count_model_parameters(tsit_model)

+------------------------------------------------+------------+
|                    Modules                     | Parameters |
+------------------------------------------------+------------+
|               backbone.pos_embed               |    160     |
|               backbone.cls_token               |     32     |
|          backbone.linear_proj.weight           |    480     |
|           backbone.linear_proj.bias            |     32     |
|   backbone.encoder.encoder.0.mha.W_Q.weight    |    1024    |
|    backbone.encoder.encoder.0.mha.W_Q.bias     |     32     |
|   backbone.encoder.encoder.0.mha.W_K.weight    |    1024    |
|    backbone.encoder.encoder.0.mha.W_K.bias     |     32     |
|   backbone.encoder.encoder.0.mha.W_V.weight    |    1024    |
|    backbone.encoder.encoder.0.mha.W_V.bias     |     32     |
| backbone.encoder.encoder.0.mha.to_out.0.weight |    1024    |
|  backbone.encoder.encoder.0.mha.to_out.0.bias  |     32     |
|  backbone.encoder.encoder.0.attn_norm.

In [4]:
# number of parameters for default HPs
tsit_model = tsai.all.TSiT(
    c_in=input_features_length,
    c_out=1,
    seq_len=seq_len,
    d_model=128,
    depth=6,
    n_heads=16,
    act='gelu',
    lsa=False,
    attn_dropout=0.3,
    dropout=0,
    drop_path_rate=0,
    mlp_ratio=1,
    qkv_bias=True,
    pre_norm=True,
    use_token=False, # use_token set to False as c_out == 1
    use_pe=True,
    cat_pos=None,
    n_cat_embeds=None,
    cat_embed_dims=None,
    cat_padding_idxs=None,
    token_size=None,
    tokenizer=None,
    feature_extractor=None,
    flatten=False,
    concat_pool=True,
    fc_dropout=0.1,
    use_bn=False,
    bias_init=None,
    y_range=None,
    custom_head=None,
)
_ = count_model_parameters(tsit_model)

+------------------------------------------------+------------+
|                    Modules                     | Parameters |
+------------------------------------------------+------------+
|               backbone.pos_embed               |    640     |
|               backbone.cls_token               |    128     |
|          backbone.linear_proj.weight           |    1920    |
|           backbone.linear_proj.bias            |    128     |
|   backbone.encoder.encoder.0.mha.W_Q.weight    |   16384    |
|    backbone.encoder.encoder.0.mha.W_Q.bias     |    128     |
|   backbone.encoder.encoder.0.mha.W_K.weight    |   16384    |
|    backbone.encoder.encoder.0.mha.W_K.bias     |    128     |
|   backbone.encoder.encoder.0.mha.W_V.weight    |   16384    |
|    backbone.encoder.encoder.0.mha.W_V.bias     |    128     |
| backbone.encoder.encoder.0.mha.to_out.0.weight |   16384    |
|  backbone.encoder.encoder.0.mha.to_out.0.bias  |    128     |
|  backbone.encoder.encoder.0.attn_norm.

### TST

In [42]:
tst_model = tsai.all.TST(
    c_in=input_features_length, 
    c_out=1,
    seq_len=seq_len,
    n_layers=1,
    d_model=32,
    n_heads=16,
    d_k=None,
    d_v=None,
    d_ff=256,
    dropout=0.1,
    act='gelu',
    fc_dropout=0.
    )
_ = count_model_parameters(tst_model)

+------------------------------------------+------------+
|                 Modules                  | Parameters |
+------------------------------------------+------------+
|                  W_pos                   |    160     |
|                W_P.weight                |    480     |
|                 W_P.bias                 |     32     |
|  encoder.layers.0.self_attn.W_Q.weight   |    1024    |
|  encoder.layers.0.self_attn.W_K.weight   |    1024    |
|  encoder.layers.0.self_attn.W_V.weight   |    1024    |
|  encoder.layers.0.self_attn.W_O.weight   |    1024    |
| encoder.layers.0.batchnorm_attn.1.weight |     32     |
|  encoder.layers.0.batchnorm_attn.1.bias  |     32     |
|       encoder.layers.0.ff.0.weight       |    8192    |
|        encoder.layers.0.ff.0.bias        |    256     |
|       encoder.layers.0.ff.3.weight       |    8192    |
|        encoder.layers.0.ff.3.bias        |     32     |
| encoder.layers.0.batchnorm_ffn.1.weight  |     32     |
|  encoder.lay

In [3]:
# number of parameters for default HPs
tst_model = tsai.all.TST(
    c_in=input_features_length, 
    c_out=1,
    seq_len=seq_len,
    n_layers=3,
    d_model=128,
    n_heads=16,
    d_k=None,
    d_v=None,
    d_ff=256,
    dropout=0.1,
    act='gelu',
    fc_dropout=0.
    )
_ = count_model_parameters(tst_model)

+------------------------------------------+------------+
|                 Modules                  | Parameters |
+------------------------------------------+------------+
|                  W_pos                   |    640     |
|                W_P.weight                |    1920    |
|                 W_P.bias                 |    128     |
|  encoder.layers.0.self_attn.W_Q.weight   |   16384    |
|  encoder.layers.0.self_attn.W_K.weight   |   16384    |
|  encoder.layers.0.self_attn.W_V.weight   |   16384    |
|  encoder.layers.0.self_attn.W_O.weight   |   16384    |
| encoder.layers.0.batchnorm_attn.1.weight |    128     |
|  encoder.layers.0.batchnorm_attn.1.bias  |    128     |
|       encoder.layers.0.ff.0.weight       |   32768    |
|        encoder.layers.0.ff.0.bias        |    256     |
|       encoder.layers.0.ff.3.weight       |   32768    |
|        encoder.layers.0.ff.3.bias        |    128     |
| encoder.layers.0.batchnorm_ffn.1.weight  |    128     |
|  encoder.lay

### XCM

In [43]:
xcm_model = tsai.all.XCM(
    c_in=input_features_length, 
    c_out=1,
    seq_len=seq_len,
    window_perc=1,
    flatten=False,
    concat_pool=False,
    fc_dropout=0,
    bn=False,
    y_range=None,
    )
_ = count_model_parameters(xcm_model)

+-------------------------+------------+
|         Modules         | Parameters |
+-------------------------+------------+
|   conv2dblock.1.weight  |    640     |
|    conv2dblock.1.bias   |    128     |
|   conv2dblock.2.weight  |    128     |
|    conv2dblock.2.bias   |    128     |
| conv2d1x1block.0.weight |    128     |
|  conv2d1x1block.0.bias  |     1      |
|   conv1dblock.0.weight  |    9600    |
|    conv1dblock.0.bias   |    128     |
|   conv1dblock.1.weight  |    128     |
|    conv1dblock.1.bias   |    128     |
| conv1d1x1block.0.weight |    128     |
|  conv1d1x1block.0.bias  |     1      |
|     conv1d.0.weight     |   10240    |
|      conv1d.0.bias      |    128     |
|     conv1d.1.weight     |    128     |
|      conv1d.1.bias      |    128     |
|     head.1.0.weight     |    128     |
|      head.1.0.bias      |     1      |
+-------------------------+------------+
Total Trainable Params: 22019


### OmniScaleCNN

In [45]:
omniscalecnn_model = tsai.all.OmniScaleCNN(
    c_in=input_features_length, 
    c_out=1,
    seq_len=seq_len,
    few_shot=False,
    layers=[8*128, 5 * 128 * 256 + 2 * 256  *128]
    )
_ = count_model_parameters(omniscalecnn_model)

+---------------------------------+------------+
|             Modules             | Parameters |
+---------------------------------+------------+
| net.0.conv_list.0.conv1d.weight |    1020    |
|  net.0.conv_list.0.conv1d.bias  |     68     |
|   net.0.conv_list.0.bn.weight   |     68     |
|    net.0.conv_list.0.bn.bias    |     68     |
| net.1.conv_list.0.conv1d.weight |   229364   |
|  net.1.conv_list.0.conv1d.bias  |    3373    |
|   net.1.conv_list.0.bn.weight   |    3373    |
|    net.1.conv_list.0.bn.bias    |    3373    |
| net.2.conv_list.0.conv1d.weight |  3453952   |
|  net.2.conv_list.0.conv1d.bias  |    1024    |
|   net.2.conv_list.0.bn.weight   |    1024    |
|    net.2.conv_list.0.bn.bias    |    1024    |
| net.2.conv_list.1.conv1d.weight |  6907904   |
|  net.2.conv_list.1.conv1d.bias  |    1024    |
|   net.2.conv_list.1.bn.weight   |    1024    |
|    net.2.conv_list.1.bn.bias    |    1024    |
|          hidden.weight          |    2048    |
|           hidden.b

### ResNet

In [47]:
resnet_model = tsai.all.ResNet(
    c_in=input_features_length, 
    c_out=1)
_ = count_model_parameters(resnet_model)

+-------------------------------+------------+
|            Modules            | Parameters |
+-------------------------------+------------+
| resblock1.convblock1.0.weight |    6720    |
| resblock1.convblock1.1.weight |     64     |
|  resblock1.convblock1.1.bias  |     64     |
| resblock1.convblock2.0.weight |   20480    |
| resblock1.convblock2.1.weight |     64     |
|  resblock1.convblock2.1.bias  |     64     |
| resblock1.convblock3.0.weight |   12288    |
| resblock1.convblock3.1.weight |     64     |
|  resblock1.convblock3.1.bias  |     64     |
|  resblock1.shortcut.0.weight  |    960     |
|  resblock1.shortcut.1.weight  |     64     |
|   resblock1.shortcut.1.bias   |     64     |
| resblock2.convblock1.0.weight |   57344    |
| resblock2.convblock1.1.weight |    128     |
|  resblock2.convblock1.1.bias  |    128     |
| resblock2.convblock2.0.weight |   81920    |
| resblock2.convblock2.1.weight |    128     |
|  resblock2.convblock2.1.bias  |    128     |
| resblock2.c